#Stary light Analysis of spectrograph using correction matrix 

In [1]:
#All dependancies  
# Need to have All files in the same folder
#Need to check if the driver is install in local machine/PC 
#https://andor.oxinst.com/downloads/view/andor-driver-pack-2.104.30065.0-(ccd,iccd-emccd)
#MS260 control the spectrograph
#AndorCCD control the CCD and data aquistion
from MS260 import MS260
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#wrapper library to control the andor ccd
from AndorCCD import AndorCCD
from AndorErrorCodes import AndorErrorCodes
import time
#controlling themonchromater
from ctypes import create_string_buffer, c_double, c_long, create_unicode_buffer, windll, byref, c_char_p,c_uint8
import ctypes as cts
from time import sleep
import sys
import csv

In [2]:
#connect to the spectrograph (mapped to com3)
spectr = MS260('COM3')
#goto grating 3
spectr.command('GRAT 3')
#goto a center wavelength of 530nm
spectr.command('gowave 530')
#define wanted ccd temperature
desiredTemp = -20
#number of time one measurment need to be taken during  performing Aqcuisition
Average_index=1
MaxWavelength=900
MiniWavelength=250
step_wavelength=4


#for raedout noise set the steps , initial time and ending time 
t1=0.01602
t2=15
step=(t2-t1)/20

try:
       ccd
except NameError:
       ccd = AndorCCD('./Andor_420OE_CC_001/','Atmcd32d.dll')
#retrieve the detectorsize (stored in ccd.pixelX and ccd.pixelY)
ccd.getDetectorSize()
#turn on the cooler of the ccd and wait untill the temperature is reached
ccd.coolerOn(desiredTemp)
cfgFile = c_char_p('./system.cfg'.encode('utf-8'))
atrFile = c_char_p('./system.atr'.encode('utf-8'))
error_report = c_char_p((' '*255).encode('utf-8'))
ret = 0
mono =  windll.LoadLibrary('./benhw32_stdcall.dll')
ret = mono.BI_report_error()
print(ret)
ret = mono.BI_build_system_model(cfgFile,error_report)
print(ret)
ret = mono.BI_report_error()
print(ret)
ret = mono.BI_load_setup(atrFile)
print(ret)
mono.BI_initialise()
mono.BI_park()
a = c_long(1)


SerialException: could not open port 'COM3': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [ ]:

# RUN this peace of code if SerialException: could not open port 'COM3': PermissionError appreare
# del spectr 

Calculation of readout Noise

In [ ]:

#closing the shutter
spectr.command('shutter c')
Readout_Noise_Measurement=[]  #2d matrix 
#As Spectrograph wavelength calibration is done in this range 
Status_flage=0
for i in np.arange(t1,t2,step):
        ccd.performAqcuisition(i,Average_index);
        #wait untill aqcuisition is complete
        while(ccd.readyForNextAqc == False):
               time.sleep(1)
        #aqcuisition is done so store the data in the dataholder
        data = ccd.data
        Readout_Noise_Measurement.append(data)
        
        Status_flage=Status_flage+1
        print("iterantion number :", Status_flage)
np.savetxt("Readout_Noise_Measurement.csv",Readout_Noise_Measurement, delimiter = ",")

Saturation time calculation

In [ ]:
#opening the shutter
spectr.command('shutter o')
saturation_time=[]  #2d matrix 
Status_flage=0
intTime=0.01602
for i in range(MiniWavelength,MaxWavelength,step_wavelength):
      flag=mono.BI_select_wavelength(c_double(i),byref(a))
      data=0
      intTime=0.016
      while(flag == -1):
            time.sleep(1)
      time.sleep(20)
      if(flag==0):
             #perform aqcuisition of an image 
            ccd.performAqcuisition(intTime,Average_index)
            while(ccd.readyForNextAqc == False):
                        time.sleep(1)
            data=ccd.data
            #perform aqcuisition of an image 
            while(np.max(data)<=0.85*65000):
                  #wait untill aqcuisition is complete
                  while(ccd.readyForNextAqc == False):
                        time.sleep(1)
                  intTime=0.95*(65000/np.max(data))*intTime
                  ccd.performAqcuisition(intTime,Average_index)
                  while(ccd.readyForNextAqc == False):
                        time.sleep(1)
                  data=ccd.data
      Status_flage=Status_flage+1
      print("iterantion number :", Status_flage)
      saturation_time.append(intTime)
np.savetxt("saturation_time.csv",saturation_time, delimiter = ",")

iterantion number : 1
iterantion number : 2
iterantion number : 3
iterantion number : 4
iterantion number : 5
iterantion number : 6
iterantion number : 7
iterantion number : 8
iterantion number : 9
iterantion number : 10
iterantion number : 11
iterantion number : 12
iterantion number : 13
iterantion number : 14
iterantion number : 15
iterantion number : 16
iterantion number : 17
iterantion number : 18
iterantion number : 19
iterantion number : 20
iterantion number : 21
iterantion number : 22
iterantion number : 23
iterantion number : 24
iterantion number : 25
iterantion number : 26
iterantion number : 27
iterantion number : 28
iterantion number : 29
iterantion number : 30
iterantion number : 31
iterantion number : 32
iterantion number : 33
iterantion number : 34
iterantion number : 35
iterantion number : 36
iterantion number : 37
iterantion number : 38
iterantion number : 39
iterantion number : 40
iterantion number : 41
iterantion number : 42
iterantion number : 43
iterantion number : 

Loading the integeartion time of signal which obtained in CSV before 

In [ ]:
with open("saturation_time.csv","r") as f:
    reader=csv.reader(f)
    data=list(reader)
intTime=np.array(data,dtype=float)


In [ ]:

Light_Measurement=[]  #2d matrix 
thermal_Noise_Measurement=[] #2d matrix
index=0
#As Spectrograph wavelength calibration is done in this range 
Status_flage=0
for i in range(MiniWavelength,MaxWavelength,step_wavelength):
      flag=mono.BI_select_wavelength(c_double(i),byref(a))
      while(flag == -1):
            time.sleep(1)
      time.sleep(20)
      if(flag==0):
        #perform aqcuisition of an image 
        spectr.command('shutter o')
        ccd.performAqcuisition(intTime[index],Average_index);
        #wait untill aqcuisition is complete
        while(ccd.readyForNextAqc == False):
               time.sleep(1)
        #aqcuisition is done so store the data in the dataholder
        data = ccd.data
        Light_Measurement.append(data)


        spectr.command('shutter c')
        ccd.performAqcuisition(intTime[index],Average_index)
        while(ccd.readyForNextAqc == False):
              time.sleep(1)
       #aqcuisition is done so store the data in the dataholde
        data = ccd.data
        thermal_Noise_Measurement.append(data)

        Status_flage=Status_flage+1
        print("iterantion number :", Status_flage)
        index=index+1

np.savetxt("thermal_Noise_Measurement.csv",thermal_Noise_Measurement, delimiter = ",")
np.savetxt("Light_Measurement.csv",Light_Measurement, delimiter = ",")


iterantion number : 1
iterantion number : 2
iterantion number : 3
iterantion number : 4
iterantion number : 5
iterantion number : 6
iterantion number : 7
iterantion number : 8
iterantion number : 9
iterantion number : 10
iterantion number : 11
iterantion number : 12
iterantion number : 13
iterantion number : 14
iterantion number : 15
iterantion number : 16
iterantion number : 17
iterantion number : 18
iterantion number : 19
iterantion number : 20
iterantion number : 21
iterantion number : 22
iterantion number : 23
iterantion number : 24
iterantion number : 25
iterantion number : 26
iterantion number : 27
iterantion number : 28
iterantion number : 29
iterantion number : 30
iterantion number : 31
iterantion number : 32
iterantion number : 33
iterantion number : 34
iterantion number : 35
iterantion number : 36
iterantion number : 37
iterantion number : 38
iterantion number : 39
iterantion number : 40
iterantion number : 41
iterantion number : 42
iterantion number : 43
iterantion number : 